In [1]:
%matplotlib notebook
import ipympl
import ROOT as r
%jsroot on
r.PyConfig.DisableRootLogon = True
r.PyConfig.IgnoreCommandLineOptions = False
import numpy as np
from array import array
import matplotlib.pyplot as plt
import pandas as pd
from array import array
import scipy.optimize as opt
from glob import glob
import time


Welcome to JupyROOT 6.28/06


## Next Step:

Now that we have a good estimate for the PA and PD, we can repeat the process using the estimated PD to get a better estimate for PA, since we have these estimates we can also search a smaller space.

In [2]:
def polfrac_model(E,PD30,beta):
    return beta*(E-30) + PD30

def polang_model(E,PA30,gamma):
    return gamma*(E-30) + PA30

In [ ]:
path_to_test_data = "~/xlc_sim_outputs/Test_EnergyDep10.root"
#path_to_test_data = "~/xlc_sim_outputs/flight_like/PD45_rotating.root"
data_set = r.RDataFrame('tree2',path_to_test_data).AsNumpy()
xbin = np.arange(0.5,33.5,1)
PA_guess,PD_guess = 40,0.4
opt_path = np.array([PA_guess,PD_guess])
psf_e_steps = [20,30,40,50,70]
start = time.time()
#vertical_likelihood_tensor = np.load(f"/home/jacob/xlc_sim_outputs/notebook_outputs/pixel-row_likelihood_tensor.npy")
def likelihoodfnc(par):
    P=0
    full_likelihood_tensor = np.load(f"/home/jacob/xlc_sim_outputs/notebook_outputs/likelihood_tensor_0_360.npy")
    #iterate through each event, toss out any with pixel>32, as these are on the bottom of xlcal
    for i in range(len(data_set['rotationangle'])):
        if (data_set['horizontal_pixel'][i] < 32):# & (data_set['pixel_row'][i] < 31):
            horizontal_pixel = data_set['horizontal_pixel'][i]

            #Get the rotation angle for each event
            rotation_angle = int(data_set['rotationangle'][i])
            
            angle = polang_model(data_set['energy'][i],par[0],par[1])
            PD = polfrac_model(data_set['energy'][i],par[2],par[3])
            #angle = par[0]
            #PD = par[1]
            
            #Get the seperation between PA and the interager abover PA that is stored in the lookup table
            x = np.ceil(angle) - angle
            
            # Get the likelihood of the polarization from array that are just above and below the expected polarization
            if angle > 180:
                angle -= 180
            elif angle < 0:
                angle += 180
            #if (PD <= 1) & (PD > 0):
            if True:
                if PD > 1:
                    PD = 1
                z1 = full_likelihood_tensor[int(np.ceil(horizontal_pixel-1)),int(np.floor(angle)),int(np.ceil(PD*100)),rotation_angle]
                z2 = full_likelihood_tensor[int(np.ceil(horizontal_pixel-1)),int(np.ceil(angle)),int(np.ceil(PD*100)),rotation_angle]
    
                z3 = full_likelihood_tensor[int(np.ceil(horizontal_pixel-1)),int(np.ceil(angle)),int(np.floor(PD*100)),rotation_angle]
                z4 = full_likelihood_tensor[int(np.ceil(horizontal_pixel-1)),int(np.floor(angle)),int(np.floor(PD*100)),rotation_angle]

                # Using the 4 nearest Polarization points, using the distance from them to the expected point as the weight, weight the likelihoods
                dx1 = int(np.floor(angle)) - angle
                dy1 = int(np.ceil(PD*100)) - (PD*100)
                a = 1/np.hypot(dx1,dy1)
                
                dx2 = int(np.ceil(angle)) - angle
                dy2 = int(np.ceil(PD*100)) - (PD*100)
                b = 1/np.hypot(dx2,dy2)
    
                dx3 = int(np.ceil(angle)) - angle
                dy3 = int(np.floor(PD*100)) - (PD*100)
                c = 1/np.hypot(dx3,dy3)
                
                dx4 = int(np.floor(angle)) - angle
                dy4 = int(np.floor(PD*100)) - (PD*100)
                d = 1/np.hypot(dx4,dy4)
                
                # weigh likelihoods with 1/hypot
                likelihood = ((a*z1)+(b*z2)+(c*z3)+(d*z4))/(a+b+c+d)
                a = -np.log(np.abs(PD*100))
                if a >= 0:
                    likelihood += a
                if PD < 0:
                    likelihood += 1000
            
                
            '''
            #When I get Energy Resolved Simulations: this is future code
            Energy = data_set['energy'][i]
            if (Energy < 70) & (Energy > 20):
                x1 = vertical_likelihood_tensor[0,int(np.ceil(data_set['pixel_row'][i])),int(np.ceil(Energy/10))]
                x2 = vertical_likelihood_tensor[0,int(np.ceil(data_set['pixel_row'][i])),int(np.floor(Energy/10))]
                
                
                dx1 = np.ceil(Energy/10) - Energy/10
                dx2 = Energy/10 - np.floor(Energy/10)
                energy_dep_vertical_pixel_likelihood = (dx1 * x1 + dx2 * x2)/(dx1+dx2)
            elif Energy >= 70:
                energy_dep_vertical_pixel_likelihood = x2 = vertical_likelihood_tensor[0,int(np.ceil(data_set['pixel_row'][i])),int(7)]
            elif Energy <= 20:
                energy_dep_vertical_pixel_likelihood = x2 = vertical_likelihood_tensor[0,int(np.ceil(data_set['pixel_row'][i])),int(2)]
            
            likelihood += energy_dep_vertical_pixel_likelihood
            '''
            P += likelihood
    return P

fitConst = False
if fitConst:
#res = opt.minimize(likelihoodfnc,np.array([45,0.1,0.45,0.1]),method='nelder-mead',bounds = ((0,180),(-5,5),(0,1),(-1,1)),options={"fatol":1E-3})
#res = opt.basinhopping(likelihoodfnc,np.array([45,35/90,0.45,0.5/90]),minimizer_kwargs={'method':'nelder-mead','bounds':((0,180),(-180/90,180/90),(0,1),(-1/90,1/90))},niter=5)
    res = opt.basinhopping(likelihoodfnc,np.array([45,0,0.45,0]),minimizer_kwargs={'method':'nelder-mead','bounds':((0,180),(-1e-7,1e-7),(0,1),(-1e-7,1e-7))},niter=5)

if not fitConst:
    res = opt.basinhopping(likelihoodfnc,np.array([50,20/90,0.3,0.2/90]),minimizer_kwargs={'method':'nelder-mead','bounds':((0,180),(-180/90,180/90),(0,1),(-1/90,1/90))},niter=3)

print(res)

end = time.time()
print(end-start)

In [ ]:
E = np.arange(10,90,1)
%matplotlib inline
fig,ax = plt.subplots(2,1,sharex = True)
ax[0].plot(E,polfrac_model(E,res.x[2],res.x[3]),label='Model')
ax[0].plot(E,polfrac_model(E,0.3,0.2/90),label='Actual')
ax[0].set_xlim(15,80)
ax[0].set_ylabel("Pol Frac [0-1]")
ax[1].set_xlabel("Energy [keV]")
ax[1].plot(E,polang_model(E,res.x[0],res.x[1]))
ax[1].plot(E,polang_model(E,50,20/90))
ax[1].set_ylabel("Pol Angle [degrees]")
#ax[0].set_ylim(0,1)
ax[0].legend()
fig.tight_layout()

## Test Code for adding Background

## Error Calculations


In [ ]:

def get_errors(par,index,error_step):
    par = par.copy()
    par2 = par.copy()
    start = time.time()
    value = res.fun
    error = 0
    par[index]
    #get the error values by incrememnting PA/PD until we get a change in the log likelihood of 1.
    while abs(res.fun - value) < 1:
        if abs(res.fun - value) < 0.25:
            error += error_step*5
        else: 
            error += error_step
        par2[index] = par[index]+error
        value = likelihoodfnc(par2)
    error -= 5*error_step
    while abs(res.fun - value) < 1:
        error += error_step/10
        par2[index] = par[index]+error
        value = likelihoodfnc(par2)
    end = time.time()
    print(end-start)
    if error == 0:
        print("smaller error on index {index}")
        error += get_errors(par,index,error_step/10)
    return error

par0e = get_errors(res.x,0,0.1)
par2e = get_errors(res.x,2,0.0001)
if not fitConst:
    par1e = get_errors(res.x,1,0.0001)
    par3e = get_errors(res.x,3,0.00001)
else:
    par1e = 0
    par3e = 0

upperbounds = [res.x[0]+par0e,res.x[1]+par1e,res.x[2]+par2e,res.x[3]+par3e]
lowerbounds = [res.x[0]-par0e,res.x[1]-par1e,res.x[2]-par2e,res.x[3]-par3e]



In [ ]:
print(upperbounds-res.x)

In [ ]:
E = np.arange(15,81,1)
%matplotlib inline
fig,ax = plt.subplots(2,1,sharex = True)
ax[0].plot(E,polfrac_model(E,res.x[2],res.x[3]),label='Model')
ax[0].plot(E,polfrac_model(E,0.3,0.2/90),label='Actual')
ax[0].fill_between(E,polfrac_model(E,lowerbounds[2],lowerbounds[3]),polfrac_model(E,upperbounds[2],upperbounds[3]),label='error',color='#ffccff')
ax[0].fill_between(E,polfrac_model(E,upperbounds[2],lowerbounds[3]),polfrac_model(E,lowerbounds[2],upperbounds[3]),color='#ffccff')

ax[0].set_ylabel("Pol Frac [0-1]")
ax[1].set_xlabel("Energy [keV]")
ax[0].set_xlim(15,80)
ax[1].plot(E,polang_model(E,res.x[0],res.x[1]))
ax[1].plot(E,polang_model(E,50,20/90))
ax[1].fill_between(E,polang_model(E,lowerbounds[0],lowerbounds[1]),polang_model(E,upperbounds[0],upperbounds[1]),label='error',color='#ffccff')
ax[1].fill_between(E,polang_model(E,upperbounds[0],lowerbounds[1]),polfrac_model(E,lowerbounds[0],upperbounds[1]),color='#ffccff')
ax[1].set_ylabel("Pol Angle [degrees]")
ax[0].legend()
fig.tight_layout()
fig.savefig('/home/jacob/Energy_dep_polarization_plots_realspec_withE.pdf')


In [ ]:

steps = 40
width = 5
if par1e == 0:
    par1e = get_errors(res.x,1,0.0001)
    par3e = get_errors(res.x,3,0.00001)
pas_ = np.linspace(res.x[0] - (par0e*width),res.x[0] + (par0e*width),steps)
pd_ = np.linspace(res.x[2] - (par2e*width),res.x[2] + (par2e*width),steps)
#pd_ = np.linspace(0.50,0.75,steps)
if pd_.max() > 1:
    pd_ = np.linspace(res.x[2] - (par2e*width),0.75,steps)
    pdmax = 0.75
else:
    pdmax = res.x[2] + (par2e*width)
if pd_.min() < 0:
    pd_ = np.linspace(0,pdmax,steps)
paslopes_ = np.linspace(res.x[1] - (par1e*width),res.x[1] + (par1e*width), steps)
pdslopes_ = np.linspace(res.x[3] - (par3e*width),res.x[3] + (par3e*width), steps)
arrays = [pas_,paslopes_,pd_,pdslopes_]
titles = ['PA_30 [degrees]','PA_slope [keV/degrees]' , 'PD_30 [0-1]', 'PD_slope [keV]' ]
fig,ax = plt.subplots(4,4,figsize=(15, 15),layout="constrained",sharex=False,sharey=False)
for i in np.arange(4):
    for j in np.arange(4):
        #print(i,j)
        par = list(res.x.copy())
        if j < i:
            
            X,Y = np.meshgrid(arrays[i],arrays[j])
            Z = np.empty_like(X)
            for k in np.arange(len(arrays[i])):
                for l in np.arange(len(arrays[j])):
                    par[i] = arrays[i][k]
                    par[j] = arrays[j][l]
                    Z[k,l] = likelihoodfnc(par)
            ax[i,j].contour(Y,X,Z.T,20,cmap='GnBu')
            #ax[i,j].set_xlim(arrays[i].min(),arrays[i].max())
            #ax[i,j].set_ylim(arrays[j].min(),arrays[j].max())
            #ax[i,j].colorbar()
            if i == 3:
                ax[i,j].set_xlabel(titles[j])
            if j == 0:
                ax[i,j].set_ylabel(titles[i])
            #if i != 0:
            #    ax[i,j].tick_params(labelleft=None)
            #if j != 0:
            #    ax[i,j].tick_params(labelbottom=None)
        elif i == j:
            Z = []
            for k in np.arange(len(arrays[i])):
                par[i] = arrays[i][k]
                Z = np.append(Z,likelihoodfnc(par))
            ax[i,j].plot(arrays[i][:],Z)
            #ax[i,j].tick_params(labelleft=None)
            if i == 3 :
                ax[i,j].set_xlabel(titles[j])
        else:
            ax[i,j].axis('off')
            continue
fig.savefig("/home/jacob/PolarizationLikelihoodErrors_realspec_smallPD.png")
#plt.show()


## Stoke's Analysis

In [ ]:
energy = data_set['energy']
constant_model = res.x.copy()
sim_model = [50,20/90,0.3,0.2/90]

def get_stokes(par,energy):
    PA = polang_model(energy,par[0],par[1])
    PD = polfrac_model(energy,par[2],par[3])

    U = PD*np.sin(2*PA*np.pi/180)
    Q = PD*np.cos(2*PA*np.pi/180)
    U_I = U.sum()/len(energy)
    Q_I = Q.sum()/len(energy)

    return U_I,Q_I



U,Q = get_stokes(sim_model,energy)
U2,Q2 = get_stokes(constant_model,energy)
table = [[U,Q,0.5*np.arctan2(U,Q)*180/np.pi,np.sqrt((U**2) + (Q**2))],[U2,Q2,0.5*np.arctan2(U2,Q2)*180/np.pi,np.sqrt((U2**2) + (Q2**2))]]
table = pd.DataFrame(table,columns=['U','Q','recalculated PA','recalculated PD'],index=['Sim Model','Fit Model'])
display(table)
PD = np.sqrt((U**2) + (Q**2))
PA = 0.5*np.arctan2(U,Q)*180/np.pi
print(1/np.sqrt(2*len(energy)))



## Old But potentially useful code